# comfyui仓库安装
> 工程仓库clone以及安装所需要的节点

In [ ]:
update = False

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt

models_path = '/kaggle/working/ComfyUI/models'

!mamba install openssh -y

custom_nodes = '/kaggle/working/ComfyUI/custom_nodes'
# install portrait-master node 
%cd $custom_nodes
!git clone https://github.com/ZHO-ZHO-ZHO/comfyui-portrait-master-zh-cn.git

# Install the node manager
update_manager = False
%cd $custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

# 模型下载

从hungingface下载模型,记得将以下的hf_*** token换成你自己的

In [ ]:

# 从hungingface下载模型, 下载方式参考文档https://padeoe.com/huggingface-large-models-downloader/

repo = 'https://huggingface.co/stabilityai/stable-diffusion-3-medium/resolve/main%s?download=true'

model_dict = {'/sd3_medium.safetensors': './checkpoints/sd3_medium.safetensors',
'/text_encoders/clip_g.safetensors':'./clip/clip_g.safetensors',
'/text_encoders/clip_l.safetensors':'./clip/clip_l.safetensors',
'/text_encoders/t5xxl_fp16.safetensors':'./clip/t5xxl_fp16.safetensors',
}

%cd $models_path

for k,v in model_dict.items():
        cmd = f'wget --header "Authorization: Bearer hf_***" {repo % k} -O {v}'
        print(cmd)
        get_ipython().system(cmd)


# 启动服务
通过浏览器打开带有.pinggy.link 后缀的链接

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

# 工作流下载

工作流json文件，从[ComfyUI-Workflows-ZHO](https://github.com/ZHO-ZHO-ZHO/ComfyUI-Workflows-ZHO) 仓库中获取SD3 Medium + 肖像大师的json文件